In [1]:
import preprocessor
import models
import numpy as np
from keras.models import model_from_json

from tqdm import tqdm

Using TensorFlow backend.


In [2]:
data = preprocessor.read_data('../train.csv')[35000:]

In [3]:
file_names = list(map(lambda x: x.name, data))
print(file_names.__len__())

5479


In [4]:
x_test = []
y_test = []

for item in tqdm(data):
    x_test.append(models.AnnotatedRecord.medium_image(item))
    y_test.append(models.AnnotatedRecord.some_hot(item))

x_data = np.array(x_test, np.float16) / 255.
y_data = np.array(y_test, np.uint8)

print(x_data.shape)
print(y_data.shape)

print("*** Loading Model ***")
# load json and create model
json_file = open('model.json', 'r')
model_json = json_file.read()
json_file.close()
model = model_from_json(model_json)

model.load_weights("model.h5")

print("*** Compiling Model ***")
model.compile(loss='binary_crossentropy',
              # We NEED binary here, since categorical_crossentropy l1 norms the output before calculating loss.
              optimizer='adam',
              metrics=['accuracy'])

print("*** Generating Predictions ***")
#scores = model.evaluate(x_train, y_train, verbose=0)
#print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

predictions = model.predict(x_data)

100%|██████████| 5479/5479 [00:03<00:00, 1799.61it/s]


(5479, 64, 64, 3)
(5479, 17)
*** Loading Model ***
*** Compiling Model ***
*** Generating Predictions ***


In [5]:
thresholds = [0.2] * len(models.inv_mapping)

predictions_labels = []
for prediction in predictions:
    labels = [models.inv_mapping[i] for i, value in enumerate(prediction) if value > thresholds[i]]
    predictions_labels.append(labels)
    
print(*predictions_labels[:10], sep='\n')

['primary', 'agriculture', 'water', 'cultivation', 'habitation', 'road', 'clear']
['primary', 'agriculture', 'water', 'cultivation', 'road', 'clear']
['primary', 'agriculture', 'water', 'cultivation', 'habitation', 'road', 'clear']
['primary', 'clear']
['primary', 'agriculture', 'cultivation', 'clear']
['primary', 'clear']
['primary', 'agriculture', 'water', 'road', 'partly_cloudy']
['primary', 'partly_cloudy']
['primary', 'partly_cloudy']
['primary', 'agriculture', 'water', 'road', 'partly_cloudy']


In [6]:
y_labels = []
for record in y_data:
    labels = [models.inv_mapping[i] for i, value in enumerate(record) if value == 1]
    y_labels.append(labels)
print(*y_labels[:10], sep='\n')

['primary', 'cultivation', 'haze', 'slash_burn']
['primary', 'agriculture', 'road', 'clear']
['primary', 'agriculture', 'water', 'cultivation', 'clear']
['primary', 'road', 'clear', 'selective_logging']
['primary', 'clear']
['primary', 'clear']
['primary', 'water', 'partly_cloudy']
['primary', 'partly_cloudy']
['primary', 'partly_cloudy']
['primary', 'agriculture', 'cultivation', 'habitation', 'road', 'partly_cloudy']


In [7]:
predict_data = []
for prediction in predictions:
    predict_data.append(list(map(lambda x: int(x > .20), prediction)))
    
print(*predict_data[:5], sep='\n')
print(*y_data[:5], sep='\n')

[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1 0 0 1 0 0 0 0 1 0 1 0 0 0 0 0 0]
[1 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0]
[1 1 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0]
[1 0 0 0 0 1 1 0 0 0 0 0 0 0 0 1 0]
[1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]


In [8]:
total_num = y_data.__len__()
print("Total number of items: {}".format(total_num))

Total number of items: 5479


In [39]:
index = 16

print("\nFeature: ", models.inv_mapping[index], "\n")

total_features = sum([x[index]==1 for x in y_data])
print("Total: ", total_features)

true_positives = sum([x[index]==1 and y[index]==1 for x, y in zip(y_data, predict_data)])
print("True Positives: ", true_positives)

false_negatives = sum([x[index]==1 and y[index]==0 for x, y in zip(y_data, predict_data)])
print("False Negatives: ", false_negatives)

false_positives = sum([x[index]==0 and y[index]==1 for x, y in zip(y_data, predict_data)])
print("False Positives: ", false_positives)

accuracy = true_positives / (true_positives + false_positives)
print("Accuracy ", accuracy)

recall = true_positives / (true_positives + false_negatives)
print("Recall ", recall)


Feature:  blow_down 

Total:  13
True Positives:  0
False Negatives:  13
False Positives:  0
Accuracy  nan
Recall  0.0


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in long_scalars
